SKRIPSI

# Import library

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import nltk
import os

from nltk import ngrams
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, validation_curve,RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from matplotlib_venn import venn2
from time import time
from joblib import dump, load
from sklearn.metrics import confusion_matrix
from sklearn import metrics

np.random.seed(0)
stopwords = nltk.corpus.stopwords.words('english')
poster_Stemmer = nltk.PorterStemmer()


# Baca file csv

In [2]:
data_train_imdb = pd.read_csv('dataset/IMDB_Train.csv')
data_test_imdb = pd.read_csv('dataset/IMDB_Test.csv')

In [3]:
print(data_train_imdb.head())

   Label                                           Sentimen
0      0  Story of a man who has unnatural feelings for ...
1      0  Airport '77 starts as a brand new luxury 747 p...
2      0  This film lacked something I couldn't put my f...
3      0  Sorry everyone,,, I know this is supposed to b...
4      0  When I was little my parents took me along to ...


In [4]:
print(data_test_imdb.head())

   Label                                           Sentimen
0      0  Once again Mr. Costner has dragged out a movie...
1      0  First of all, I would like to say that I am a ...
2      0  I'm a huge fan of both Emily Watson (Breaking ...
3      0  I was pulled into this movie early on, much to...
4      0  This tale of the upper-classes getting their c...


# Preprocessing 

In [5]:
data_Preprocessing_IMDB_Train = data_train_imdb.Sentimen.astype(str)
data_Preprocessing_IMDB_Train = data_Preprocessing_IMDB_Train.apply(lambda x: x.lower())
data_Preprocessing_IMDB_Train = data_Preprocessing_IMDB_Train.apply(lambda x: re.sub(r"\d", "", x))   
data_Preprocessing_IMDB_Train = data_Preprocessing_IMDB_Train.apply(
                                    lambda x: x.translate(str.maketrans('','',string.punctuation)))  
data_Preprocessing_IMDB_Train = data_Preprocessing_IMDB_Train.apply(
                                    lambda x: " ".join(re.findall("[a-zA-Z]+", x)))

In [6]:
data_Preprocessing_IMDB_Train.head()

0    story of a man who has unnatural feelings for ...
1    airport starts as a brand new luxury plane is ...
2    this film lacked something i couldnt put my fi...
3    sorry everyone i know this is supposed to be a...
4    when i was little my parents took me along to ...
Name: Sentimen, dtype: object

In [7]:
data_Preprocessing_IMDB_Test = data_test_imdb.Sentimen.astype(str)
data_Preprocessing_IMDB_Test = data_Preprocessing_IMDB_Test.apply(lambda x: x.lower())
data_Preprocessing_IMDB_Test = data_Preprocessing_IMDB_Test.apply(lambda x: re.sub(r"\d", "", x))   
data_Preprocessing_IMDB_Test = data_Preprocessing_IMDB_Test.apply(lambda x: x.translate(
                                str.maketrans('','',string.punctuation)))  
data_Preprocessing_IMDB_Test = data_Preprocessing_IMDB_Test.apply(lambda x: " ".join(
                                re.findall("[a-zA-Z]+", x)))

In [8]:
data_Preprocessing_IMDB_Test.head()

0    once again mr costner has dragged out a movie ...
1    first of all i would like to say that i am a f...
2    im a huge fan of both emily watson breaking th...
3    i was pulled into this movie early on much to ...
4    this tale of the upperclasses getting their co...
Name: Sentimen, dtype: object

In [9]:
Clean_IMDB_Sentiment_Train = data_Preprocessing_IMDB_Train
Label_IMDB_Train = data_train_imdb['Label']
Clean_IMDB_Sentiment_Test = data_Preprocessing_IMDB_Test
Label_IMDB_Test = data_test_imdb['Label']

# N-Gram (bigram dan trigram)

In [10]:
def bigram_trigram(Clean_Sentiment) :
    iterator = 0
    Vocabulary = []
    ngram_result = []
    while iterator < len(Clean_Sentiment) :
        ngram = 2
        sentence = Clean_Sentiment[iterator]
        vocab = list(ngrams(sentence, ngram))
        temp = vocab

        iterator2 = 0
        while iterator2 < len (vocab):
            if(vocab[iterator2] not in Vocabulary) :
                Vocabulary.append(vocab[iterator2])
            iterator2 = iterator2 + 1

        ngram = 3
        vocab = list(ngrams(sentence, ngram))
        temp = temp + vocab
        ngram_result.append(temp)
        iterator2 = 0
        while iterator2 < len (vocab):
            if(vocab[iterator2] not in Vocabulary) :
                Vocabulary.append(vocab[iterator2])
            iterator2 = iterator2 + 1
        iterator = iterator + 1
    
    return ngram_result,Vocabulary


In [11]:
ngram_imdb_train, Vocabulary_IMDB_Train = bigram_trigram(Clean_IMDB_Sentiment_Train)

In [12]:
ngram_imdb_test, Vocabulary_IMDB_Test = bigram_trigram(Clean_IMDB_Sentiment_Test)

# Me inisialisasi jumlah frekuensi Gram di Vocabulary

In [13]:
def dictionaryInitialize(Clean_Sentiment, Vocabulary, ngram, jenisdata) :
    wordDictionaryCount = []
    iterator = 0
    while iterator < len(Clean_Sentiment):
        wordDictionaryCount.append(dict.fromkeys(Vocabulary , 0))
        iterator = iterator + 1
    
    iterator = 0
    
    while iterator < len(Clean_Sentiment) :
        for gram in ngram[iterator] :
            if (jenisdata == 'test' or jenisdata == 'transfer') and gram not in Vocabulary :
                continue
            wordDictionaryCount[iterator][gram] += 1
        iterator = iterator + 1
    
    return wordDictionaryCount

In [14]:
WordDictionaryCount_IMDB_Train = dictionaryInitialize(Clean_IMDB_Sentiment_Train, Vocabulary_IMDB_Train, ngram_imdb_train,'train')

In [15]:
WordDictionaryCount_IMDB_Test = dictionaryInitialize(Clean_IMDB_Sentiment_Test, Vocabulary_IMDB_Train, ngram_imdb_test,'test')

In [16]:
pd.DataFrame([WordDictionaryCount_IMDB_Train[0],WordDictionaryCount_IMDB_Train[1]])

,"(s, t)","(t, o)","(o, r)","(r, y)","(y, )","( , o)","(o, f)","(f, )","( , a)","(a, )",...,"(k, b, l)","(d, m, y)","(p, s, n)","(g, p, i)","( , l, w)","(w, r, )","(t, t, t)","(u, a, x)","(h, b, y)","(w, o, f)"
0,7,5,6,2,10,9,5,5,12,9,...,0,0,0,0,0,0,0,0,0,0
1,25,19,47,5,33,65,25,29,81,28,...,0,0,0,0,0,0,0,0,0,0


## Menghitung nilai TF pada setiap data 

In [17]:
def computeFrequency(wordDict, ngram):
    freqword = {}
    ngramLength = len(ngram)
    for word, count in wordDict.items():
        freqword[word] = count
    return freqword

In [18]:
def getFrequencyTermPerData(wordDictionary, ngram) :
    CountFrequency = []
    iterator = 0
    while iterator < len(ngram):
        CountFrequency.append(computeFrequency(wordDictionary[iterator], ngram[iterator]))
        iterator = iterator + 1
    
    return CountFrequency

In [19]:
CountFrequency_Sentimen_IMDB_Train = getFrequencyTermPerData(WordDictionaryCount_IMDB_Train, ngram_imdb_train)
CountFrequency_Sentimen_IMDB_Test = getFrequencyTermPerData(WordDictionaryCount_IMDB_Test, ngram_imdb_test)

## Menghitung nilai IDF pada setiap data Amazon dan Yelp

In [20]:
def computeIDF(WordDict):
    idfDict = {}
    number_of_document_with_term_t_in_it = {}
    N = len(WordDict)
    
    idfDict = dict.fromkeys(WordDict[0].keys(), 0)
    for doc in WordDict:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        number_of_document_with_term_t_in_it[word] = float(val)
        idfDict[word] = np.log((1+N) / (1+val)) + 1
        
    return idfDict, number_of_document_with_term_t_in_it

In [21]:
IDF_IMDB, number_of_document_with_term_t_in_IMDB_Train = computeIDF(WordDictionaryCount_IMDB_Train)

In [22]:
def computeTFIDFnonScale(CountFrequency, idfs):
    tfidf = None
    tfidf = {}
    for word, val in CountFrequency.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [23]:
def computeTFIDF(tfidf_nonscaled) :
    tfidf_nonscaled = np.array(tfidf_nonscaled)
    tfidf_list = tfidf_nonscaled/sum(tfidf_nonscaled**2)**0.5
    return tfidf_list

In [24]:
def getTFIDF(wordDictionary, CountFrequency, IDF):
    TFIDF_non_scaled = None
    TFIDF_non_scaled = []
    iterator = 0
    while iterator < len(wordDictionary):
        TFIDF_non_scaled.append(computeTFIDFnonScale(CountFrequency[iterator], IDF))
        iterator = iterator + 1
    
    TFIDF = []
    iterator = 0
    while iterator < len(wordDictionary):
        tf_idf_list = list(TFIDF_non_scaled[iterator].values())
        TFIDF.append(computeTFIDF(tf_idf_list))
        iterator = iterator + 1
        
    return TFIDF

In [25]:
TFIDF_IMDB_Train = getTFIDF(WordDictionaryCount_IMDB_Train, CountFrequency_Sentimen_IMDB_Train, IDF_IMDB)

In [26]:
TFIDF_IMDB_Test = getTFIDF(WordDictionaryCount_IMDB_Test, CountFrequency_Sentimen_IMDB_Test, IDF_IMDB)

In [27]:
def toVectorizeList(TFIDF):
    iterator  = 0
    Vectorized = []
    while iterator < len(TFIDF):
        Vectorized.append(list(TFIDF[iterator]))
        iterator = iterator + 1
    return Vectorized

In [28]:
Vectorizer_IMDB_Train = toVectorizeList(TFIDF_IMDB_Train)
Vectorizer_IMDB_Test = toVectorizeList(TFIDF_IMDB_Test)

In [29]:
DataFrame_IMDB_Test = pd.DataFrame(Vectorizer_IMDB_Test, columns = Vocabulary_IMDB_Train)
DataFrame_IMDB_Test = DataFrame_IMDB_Test.reindex(sorted(DataFrame_IMDB_Test.columns), axis=1)
print (DataFrame_IMDB_Test)

        ( , a)  ( , a,  )  ( , a, a)  ( , a, b)  ( , a, c)  ( , a, d)  \
0     0.179009   0.023233        0.0   0.037041   0.000000   0.000000   
1     0.167577   0.082649        0.0   0.006935   0.006172   0.000000   
2     0.188543   0.027573        0.0   0.021979   0.015649   0.012406   
3     0.117361   0.012186        0.0   0.019427   0.000000   0.013707   
4     0.159632   0.042972        0.0   0.000000   0.008710   0.027621   
...        ...        ...        ...        ...        ...        ...   
1245  0.207539   0.068565        0.0   0.000000   0.000000   0.000000   
1246  0.153714   0.050698        0.0   0.002994   0.007993   0.000000   
1247  0.178842   0.039094        0.0   0.000000   0.013867   0.000000   
1248  0.191903   0.048468        0.0   0.008586   0.000000   0.000000   
1249  0.212450   0.042017        0.0   0.000000   0.000000   0.023631   

      ( , a, e)  ( , a, f)  ( , a, g)  ( , a, h)  ...  (z, y, s)  (z, z)  \
0           0.0   0.000000   0.017111   0.00000

## Akurasi dan Waktu Train Test 

In [30]:
Waktu_Training = time()
RF_Classifier_IMDB = RandomForestClassifier(max_depth= 5, 
                    n_estimators = 800, random_state=42,
                    bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_IMDB.fit(Vectorizer_IMDB_Train, Label_IMDB_Train)
print(f"\nWaktu Training IMDB: {round(time()-Waktu_Training, 3)}s")


Waktu Training IMDB: 31.136s


In [31]:
Waktu_Predict_Train = time()
Skor_Train_IMDB = RF_Classifier_IMDB.score(Vectorizer_IMDB_Train, Label_IMDB_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_IMDB = RF_Classifier_IMDB.score(Vectorizer_IMDB_Test, Label_IMDB_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train IMDB : {}".format(Skor_Train_IMDB))
print("Skor Random Forest Test IMDB : {}".format(Skor_Test_IMDB))

waktu prediksi (train): 2.843s
waktu prediksi (test): 0.98s

Skor Random Forest Train IMDB : 0.9157333333333333
Skor Random Forest Test IMDB : 0.7816


In [32]:
Waktu_Predict_Train = time()
Skor_Train_IMDB = RF_Classifier_IMDB.score(Vectorizer_IMDB_Train, Label_IMDB_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_IMDB = RF_Classifier_IMDB.score(Vectorizer_IMDB_Test, Label_IMDB_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest: {}".format(Skor_Train_IMDB))
print("Skor Random Forest: {}".format(Skor_Test_IMDB))
print("----------------------------------------------------\n\n")

RFClassifier_predict = RF_Classifier_IMDB.predict(Vectorizer_IMDB_Train)
Confusion_matrix = confusion_matrix(Label_IMDB_Train, RFClassifier_predict)
print(Confusion_matrix)
print(metrics.classification_report(Label_IMDB_Train, RFClassifier_predict))

print("----------------------------------------------------\n\n")
RFClassifier_predict = RF_Classifier_IMDB.predict(Vectorizer_IMDB_Test)
Confusion_matrix = confusion_matrix(Label_IMDB_Test, RFClassifier_predict)
print(Confusion_matrix)
print(metrics.classification_report(Label_IMDB_Test, RFClassifier_predict))

waktu prediksi (train): 2.882s
waktu prediksi (test): 0.959s

Skor Random Forest: 0.9157333333333333
Skor Random Forest: 0.7816
----------------------------------------------------


[[1698  177]
 [ 139 1736]]
              precision    recall  f1-score   support

           0       0.92      0.91      0.91      1875
           1       0.91      0.93      0.92      1875

    accuracy                           0.92      3750
   macro avg       0.92      0.92      0.92      3750
weighted avg       0.92      0.92      0.92      3750

----------------------------------------------------


[[471 154]
 [119 506]]
              precision    recall  f1-score   support

           0       0.80      0.75      0.78       625
           1       0.77      0.81      0.79       625

    accuracy                           0.78      1250
   macro avg       0.78      0.78      0.78      1250
weighted avg       0.78      0.78      0.78      1250



In [33]:
from joblib import dump, load

In [35]:
Vocabulary_Importance_Amazon = load('savedlist/vocabularyImportanceAmazon')

In [36]:
Vocabulary_intersection = list(set(
                            Vocabulary_Importance_Amazon) & set (
                            Vocabulary_IMDB_Train))

In [37]:
WordDictionary_feature_importance_Train = dictionaryInitialize(
            Clean_IMDB_Sentiment_Train,
            Vocabulary_intersection, 
            ngram_imdb_train,'transfer')

In [38]:
pd.DataFrame([WordDictionaryCount_IMDB_Train[0],WordDictionaryCount_IMDB_Train[1]])

,"(s, t)","(t, o)","(o, r)","(r, y)","(y, )","( , o)","(o, f)","(f, )","( , a)","(a, )",...,"(k, b, l)","(d, m, y)","(p, s, n)","(g, p, i)","( , l, w)","(w, r, )","(t, t, t)","(u, a, x)","(h, b, y)","(w, o, f)"
0,7,5,6,2,10,9,5,5,12,9,...,0,0,0,0,0,0,0,0,0,0
1,25,19,47,5,33,65,25,29,81,28,...,0,0,0,0,0,0,0,0,0,0


In [39]:
pd.DataFrame([WordDictionary_feature_importance_Train[0],WordDictionary_feature_importance_Train[1]])

,"(z, e, )","(h, n)","( , k)","(o, n, a)","(h, e, a)","(e, m)","(c, , a)","(d, n, t)","(n, , c)","(c, o, p)",...,"(e, n, c)","(e, g, y)","(i, l, t)","(r, k, )","(a, u, g)","(l, u, e)","(r, u, l)","(l, u, d)","(y, a)","(t, , u)"
0,0,1,1,0,0,2,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,6,0,1,5,1,2,2,2,...,2,0,0,1,1,1,0,2,0,0


In [40]:
WordDictionary_feature_importance_Test = dictionaryInitialize(
            Clean_IMDB_Sentiment_Test, 
            Vocabulary_intersection, 
            ngram_imdb_test,'test')

In [41]:
CountFrequency_Sentimen_Train_FI = getFrequencyTermPerData(WordDictionary_feature_importance_Train, ngram_imdb_train)

In [42]:
CountFrequency_Sentimen_Test_FI = getFrequencyTermPerData(WordDictionary_feature_importance_Test, ngram_imdb_test)

In [43]:
IDF_Feature_Importance, number_term  = computeIDF(WordDictionary_feature_importance_Train)

In [44]:
def getTFIDF(wordDictionary, CountFrequency, IDF):
    TFIDF_non_scaled = None
    TFIDF_non_scaled = []
    iterator = 0
    while iterator < len(wordDictionary):
        TFIDF_non_scaled.append(computeTFIDFnonScale(CountFrequency[iterator], IDF))
        iterator = iterator + 1
    
    TFIDF = []
    iterator = 0
    while iterator < len(wordDictionary):
        tf_idf_list = list(TFIDF_non_scaled[iterator].values())
        TFIDF.append(computeTFIDF(tf_idf_list))
        iterator = iterator + 1
        
    return TFIDF

In [45]:
TFIDF_Feature_importance_Train = getTFIDF(
        WordDictionary_feature_importance_Train, 
        CountFrequency_Sentimen_Train_FI, 
        IDF_Feature_Importance)

In [46]:
TFIDF_Feature_importance_Test = getTFIDF(
        WordDictionary_feature_importance_Test, 
        CountFrequency_Sentimen_Test_FI, 
        IDF_Feature_Importance)

In [47]:
Vectorizer_TF_Train = toVectorizeList(TFIDF_Feature_importance_Train)
TFIDF_Feature_importance_Train = Vectorizer_TF_Train

In [48]:
Vectorizer_TF_Test = toVectorizeList(TFIDF_Feature_importance_Test)
TFIDF_Feature_importance_Test = Vectorizer_TF_Test

In [49]:
Waktu_Training = time()
RF_Classifier_TF_FI = RandomForestClassifier(
        max_depth= 5, n_estimators = 800, random_state=42,
        bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_TF_FI.fit(TFIDF_Feature_importance_Train, Label_IMDB_Train)
print(f"\nWaktu Training: {round(time()-Waktu_Training, 3)}s")


Waktu Training: 27.174s


In [50]:
Waktu_Predict_Train = time()
Skor_Train_Interseksi_IMDB = RF_Classifier_TF_FI.score(TFIDF_Feature_importance_Train, Label_IMDB_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_Interseksi_IMDB = RF_Classifier_TF_FI.score(TFIDF_Feature_importance_Test, Label_IMDB_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train Interseksi : {}".format(Skor_Train_Interseksi_IMDB))
print("Skor Random Forest Test Interseksi : {}".format(Skor_Test_Interseksi_IMDB))
print("----------------------------------------------------\n\n")

RFClassifier_predict = RF_Classifier_TF_FI.predict(TFIDF_Feature_importance_Train)
Confusion_matrix = confusion_matrix(Label_IMDB_Train, RFClassifier_predict)
print(Confusion_matrix)
print(metrics.classification_report(Label_IMDB_Train, RFClassifier_predict))

print("----------------------------------------------------\n\n")
RFClassifier_predict = RF_Classifier_TF_FI.predict(TFIDF_Feature_importance_Test)
Confusion_matrix = confusion_matrix(Label_IMDB_Test, RFClassifier_predict)
print(Confusion_matrix)
print(metrics.classification_report(Label_IMDB_Test, RFClassifier_predict))

waktu prediksi (train): 1.198s
waktu prediksi (test): 0.416s

Skor Random Forest Train Interseksi : 0.9010666666666667
Skor Random Forest Test Interseksi : 0.7632
----------------------------------------------------


[[1656  219]
 [ 152 1723]]
              precision    recall  f1-score   support

           0       0.92      0.88      0.90      1875
           1       0.89      0.92      0.90      1875

    accuracy                           0.90      3750
   macro avg       0.90      0.90      0.90      3750
weighted avg       0.90      0.90      0.90      3750

----------------------------------------------------


[[456 169]
 [127 498]]
              precision    recall  f1-score   support

           0       0.78      0.73      0.75       625
           1       0.75      0.80      0.77       625

    accuracy                           0.76      1250
   macro avg       0.76      0.76      0.76      1250
weighted avg       0.76      0.76      0.76      1250

